In [1]:
import os
import json
import pandas as pd
import pytz
from datetime import datetime
from sqlalchemy import create_engine


In [2]:
# 접속 정보
host = "10.28.224.177"
port = 30634
database = "postgres"
username = "postgres"
password = "0104"

In [3]:
# SQLAlchemy 엔진 생성
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# 유튜브 데이터 DB에 저장

### channel

In [4]:
channel_lst = ["피식대학", "SGBG", "swab85", "konuri", "Yoiki", "NERDULT", "donghahahaha", "JBKWAK", "geean84", "carthejungwon"]
channel_name = ['피식대학', '싱긍벙글', '승우아빠', '고누리', '가요이', '너덜트', '동하하', '곽튜브', '기안84', '카더정원']

In [5]:
columns_name = ['id', 'name' , 'uId', 'title', 'description', 'publishedAt', 'thumbnails', 'viewCount', 'subscriberCount', 'videoCount', 'topicCategories', 'dataTime']

In [6]:
# 한국 시간대 설정
korea_timezone = pytz.timezone('Asia/Seoul')

# 현재 시간 가져오기
korea_time = datetime.now(korea_timezone)

# 원하는 형식으로 출력 (YYYY-MM-DD)
formatted_time = korea_time.strftime('%Y-%m-%d')

In [23]:
result = []
for name, channel in zip(channel_name, channel_lst):
    with open('./'+ channel + '/' + 'chanel_info.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(data)
    items = data['items']
    uId = items[0]['id']
    title = items[0]['snippet']['title']
    description = items[0]['snippet']['description']
    publishedAt = items[0]['snippet']['publishedAt']
    thumbnails = items[0]['snippet']['thumbnails']['default']['url']
    viewCount = items[0]['statistics']['viewCount']
    subscriberCount = items[0]['statistics']['subscriberCount']
    videoCount = items[0]['statistics']['videoCount']
    topicCategories = items[0]['topicDetails']['topicCategories']
    dataTime = formatted_time
    result.append([name, channel, uId, title, description, publishedAt, thumbnails, viewCount, subscriberCount, videoCount, topicCategories, dataTime])

{'kind': 'youtube#channelListResponse', 'etag': 'LXVXij--1XO-PEvmvyJd-aHX-Uc', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'X0MC8wQC67nrc7tnpHRGlbgScQQ', 'id': 'UCGX5sP4ehBkihHwt5bs5wvg', 'snippet': {'title': '피식대학Psick Univ', 'description': '코미디 인재 육성 및 연구의 메카 피식대학 \n\n✅ 피식대학 비즈니스 메일\npsickuniv@metacomedy.net\n\n✅ 피식대학 사서함\n(04054) 서울특별시 마포구 와우산로 29가길 42, 메타코미디\n\n✅ 피식대학 굿즈 스토어\nhttps://smartstore.naver.com/psickuniv\n', 'customUrl': '@피식대학', 'publishedAt': '2019-04-01T05:31:41Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1OnV8wL-9IzC4aRr52M6q4z8I=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1OnV8wL-9IzC4aRr52M6q4z8I=s240-c-k-c0x00ffffff-no-rj', 'width': 240, 'height': 240}, 'high': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1On

In [26]:
df_c = pd.DataFrame(result)

In [29]:
df_c = df_c.reset_index()
df_c.columns = columns_name

In [31]:
df_c.to_csv('channel_v1.csv', index=False)

In [41]:
table_name = "Channel"  # 저장할 테이블 이름
df_c.to_sql(table_name, engine, if_exists="replace", index=False)

10

### Video

In [72]:
columns_name = ['channel_id', 'vId','videoTitle', 'videoDescription', 'videoPublishedAt', 'videoThumbnails', 'tags', 'categoryId', 'liveBroadcastContent', 'videoViewCount', 'videoLikeCount', 'videoDislikeCount', 'commentCount', 'topicDetails', 'hasPaidProductPlacement']

In [73]:
result = []
for channel_id, channel in zip(range(len(channel_lst)), channel_lst):
    files = os.listdir(channel)
    files = [x for x in files if x.find('video_') != -1]
    for f in files:
        with open('./'+ channel + '/' + f, 'r', encoding='utf-8') as file:
            data = json.load(file)
        # print(data)
        itmes = data['items']
        vId = itmes[0]['id']
        videoTitle = itmes[0]['snippet']['title']
        videoDescription = itmes[0]['snippet']['description']
        videoPublishedAt = itmes[0]['snippet']['publishedAt']
        videoThumbnails = itmes[0]['snippet']['thumbnails']['default']['url']
        tags = itmes[0]['snippet'].get('tags')
        categoryId = itmes[0]['snippet']['categoryId']
        liveBroadcastContent = itmes[0]['snippet']['liveBroadcastContent']
        
        videoViewCount = itmes[0]['statistics']['viewCount']
        videoLikeCount = itmes[0]['statistics'].get('likeCount')
        # print(videoLikeCount)
        videoDislikeCount = itmes[0]['statistics'].get('dislikeCount')
        commentCount = itmes[0]['statistics'].get('commentCount')
        topicDetails = itmes[0].get('topicDetails')
        if topicDetails:
            topicDetails = topicDetails['topicCategories']
        
        hasPaidProductPlacement = itmes[0]['paidProductPlacementDetails']['hasPaidProductPlacement']
            
        result.append([channel_id, vId, videoTitle, videoDescription, videoPublishedAt, videoThumbnails, tags,
                       categoryId, liveBroadcastContent, videoViewCount, videoLikeCount, 
                       videoDislikeCount, commentCount, topicDetails, hasPaidProductPlacement])
    

In [74]:
df_v = pd.DataFrame(result, columns=columns_name)

In [75]:
df_v = df_v.sort_values(['channel_id', 'videoPublishedAt'])

In [76]:
len(df_v)

2191

In [78]:
df_v.value_counts('hasPaidProductPlacement')

hasPaidProductPlacement
False    1826
True      365
Name: count, dtype: int64

In [77]:
df_v[df_v['vId'] == '__fPyM34Cik']

,channel_id,vId,videoTitle,videoDescription,videoPublishedAt,videoThumbnails,tags,categoryId,liveBroadcastContent,videoViewCount,videoLikeCount,videoDislikeCount,commentCount,topicDetails,hasPaidProductPlacement
278,0,__fPyM34Cik,2024 패션왕 선발대회 결승전｜김해준 vs 이선민,#피식대학 #긁 \n\n김해준 : https://www.instagram.com/h...,2024-08-04T10:03:00Z,https://i.ytimg.com/vi/__fPyM34Cik/default.jpg,None,23,none,847447,10814,None,833,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,False


In [79]:
df_v.to_sql('Video', engine, if_exists="replace", index=False)

191

### Comments

In [8]:
channel_name

['피식대학', '싱긍벙글', '승우아빠', '고누리', '가요이', '너덜트', '동하하', '곽튜브', '기안84', '카더정원']

In [15]:
columns_name = ['channelName','vId', 'cId', 'textDisplay', 'authorDisplayName', 'likeCount', 'commentPublishedAt', 'commentUpdatedAt', 'replyCount', 'replies']

In [13]:
result = []
for channel_id, name, channel in zip(range(len(channel_lst)), channel_name, channel_lst):
    files = os.listdir(channel)
    files = [x for x in files if x.find('comment_') != -1]
    for f in files:
        try:
            with open('./'+ channel + '/' + f, 'r', encoding='utf-8') as file:
                data = json.load(file)
        except:
            continue
        itmes = data['items']
        
        for item in itmes:
            vId = item['snippet']['videoId']
            cId = item['id']
            
            textDisplay = item['snippet']['topLevelComment']['snippet']['textDisplay']
            authorDisplayName = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            likeCount = item['snippet']['topLevelComment']['snippet']['likeCount']
            commentPublishedAt = item['snippet']['topLevelComment']['snippet']['publishedAt']
            commentUpdatedAt = item['snippet']['topLevelComment']['snippet']['updatedAt']
            replyCount = item['snippet']['totalReplyCount']
            
            replies_lst = item.get('replies')
            if replies_lst:
                replies = [[x['snippet']['textDisplay'], x['snippet']['authorDisplayName']] for x in replies_lst['comments']]
            else:
                replies = None
                
            result.append([name, vId, cId, textDisplay, authorDisplayName, likeCount, commentPublishedAt, commentUpdatedAt, replyCount, str(replies)])

In [14]:
len(result)

580109

In [16]:
df_co = pd.DataFrame(result, columns=columns_name)

In [17]:
df_co = df_co.sort_values(['vId', 'likeCount'], ascending=False)

In [18]:
len(df_co)

580109

In [19]:
df_co.head(3)

,channelName,vId,cId,textDisplay,authorDisplayName,likeCount,commentPublishedAt,commentUpdatedAt,replyCount,replies
365962,가요이,zygTBxFDt38,UgyHwdQ9q3aThng3X7l4AaABAg,형들아 담배나 피러 가자,@이한얼-h2o,440,2020-12-05T09:42:26Z,2020-12-05T09:42:26Z,16,"[['에세 새로 나온거나 펴야지 ㄸㅂ..', '@김-f1w'], ['가자..', '..."
365958,가요이,zygTBxFDt38,UgxdEo0FKUHKhm_nVIN4AaABAg,가요이님은 진짜 매력으로 따지면 다시나올수없는 늪이고 진심 인간비타민이심ㅋㅋㅋ,@eastsea122,401,2020-12-04T10:59:59Z,2020-12-04T10:59:59Z,1,"[['일본이 질투하고 미국이 놀라는 그녀의 상큼함 ㅠㅠㅠㅠㅠ', '@김범-f3l']]"
365963,가요이,zygTBxFDt38,UgzOwCMbGoivURo97214AaABAg,진짜 대한민국 여자친구 중에 레전드<br>국민 여동생 문근영? <br>그딴거 필요없...,@삼성연합,379,2020-12-04T15:17:36Z,2020-12-04T15:18:03Z,7,"[['굳이 누가 필요없다 이딴말을 해야하누 ㅋㅋ', '@반박시니말이맞음-u6c'],..."


In [20]:
df_co.to_sql('Comments', engine, if_exists="replace", index=False)

109

In [67]:
query = f'SELECT * FROM public."Comments"'
comments_df = pd.read_sql(query, engine)

In [68]:
for _, row in comments_df.iterrows():
    print(eval(row['replies']))
    break

[['에세 새로 나온거나 펴야지 ㄸㅂ..', '@김-f1w'], ['가자..', '@화정동레킹볼'], ['전담핀다', '@merem5336'], ['가즈아. .   신기루같은거야', '@donrich5908'], ['두개 연짝으로 피고옴', '@키데니']]
